## Homework 2
Adam Lim, 5/11/19 <br></br>
Exercises from the course Learning From Data: https://work.caltech.edu/homeworks.html
<br></br>
Homework 2 link: https://work.caltech.edu/homework/hw2.pdf

### Questions #1-2
Run a computer simulation for flipping 1,000 virtual fair coins. Flip each coin independently 10 times. Focus on 3 coins as follows: c1 is the first coin flipped, crand is a coin chosen randomly from the 1,000, and cmin is the coin which had the minimum frequency of heads (pick the earlier one in case of a tie). Let ν1, νrand, and νmin be the fraction of heads obtained for the 3 respective coins out of the 10 tosses. Run the experiment 100,000 times in order to get a full distribution of ν1, νrand, and νmin (note that crand and cmin will change from run to run).

<b> 1. </b> The average value of νmin is closest to:<br></br>
<b> 2. </b> Which coin(s) has a distribution of ν that satisfies the (single-bin) Hoeffding Inequality?<br></br>

In [1]:
import pandas as pd
import numpy as np
import random
import math
np.random.seed(1234)

In [2]:
%%time

v1_distribution = []
vmin_distribution = []
vrand_distribution = []

for i in range(0, 100000):
    #print('***************** TRIAL {} *****************'.format(i))
    df = pd.DataFrame(np.random.randint(0,2, size=(1000,10)))
    df['num_heads'] = df.sum(axis=1)

    v1 = df.iloc[0]['num_heads']
    vmin = df.iloc[df['num_heads'].idxmin(axis=1)]['num_heads']
    vrand = df.sample(1)['num_heads'].values[0]

    #print('V1: {}'.format(v1))
    #print('Vmin: {}'.format(vmin))
    #print('Vrand: {}'.format(vrand))

    v1_distribution.append(v1)
    vmin_distribution.append(vmin)
    vrand_distribution.append(vrand)

CPU times: user 4min 55s, sys: 1.23 s, total: 4min 57s
Wall time: 5min 3s


In [3]:
avg_v1 = sum(v1_distribution) / (100000*10)
print(avg_v1)

avg_vmin = sum(vmin_distribution) / (100000*10)
print(avg_vmin)

avg_vrand = sum(vrand_distribution) / (100000*10)
print(avg_vrand)

0.499499
0.037599
0.500145


In [4]:
def hoeffding(e_in):
    epsilon = 0.5-e_in
    probability = math.pow(math.e, -2*math.pow(epsilon,2)*10000)
    return probability

In [5]:
print(hoeffding(avg_v1))
print(hoeffding(avg_vmin))
print(hoeffding(avg_vrand))

0.9949925592422487
0.0
0.9995795883977344


<b> Answer (1) </b> <br></br>
<b>[b]</b> 0.01 <br></br>

<b> Answer (2) </b> <br></br>
<b>[d]</b> c1 and crand <br></br>

### Questions # 3-4
Error and Noise
Consider the bin model for a hypothesis h that makes an error with probability µ in approximating a deterministic target function f (both h and f are binary functions). If we use the same h to approximate a noisy version of f given by:

<b>3.</b> What is the probability of error that h makes in approximating y? Hint: Two wrongs can make a right! <br></br>

h(x) approximates f(x); f(x) approximates y.

| P(h(x)) | h(x) ≃ f(x)  |
|---------|--------------|
| μ       | h(x) != f(x) |
| 1 - μ   | h(x) = f(x)  |

<br></br>

| P(f(x)) | f(x) ≃ y  |
|---------|-----------|
| λ       | y = f(x)  |
| 1 - λ   | y != f(x) |

<br></br>
h(x) approximates f correctly when a) h(x) correctly approximates f(x) and f(x) correctly approximates y or b) h(x) incorrectly approximates f(x) and f(x) incorrectly approximates y. Thus, h incorrectly approximates y with probability: `(μ * λ) + ((1 - μ) * (1 - λ))` <br></br>

<b> Answer </b> <br></br>
<b>[e]</b> `(1 − λ) ∗ (1 − µ) + (λ ∗ µ)`

<b>4.</b> At what value of λ will the performance of h be independent of µ? <br></br>
`(1 − λ) ∗ (1 − µ) + λ ∗ µ` = `2µλ + 1 - µ - λ`. When `λ = 0.5`, this simplifies to 0.5 <br></br>

<b> Answer </b><br></br>
<b>[b]</b> 0.5

### Questions # 5-7
<b> Linear Regression </b> <br></br>
In these problems, we will explore how Linear Regression for classification works. As with the Perceptron Learning Algorithm in Homework # 1, you will create your own target function f and data set D. Take d = 2 so you can visualize the problem, and assume X = [−1, 1] × [−1, 1] with uniform probability of picking each x ∈ X . In each run, choose a random line in the plane as your target function f (do this by taking two random, uniformly distributed points in [−1,1] × [−1, 1] and taking the line passing through them), where one side of the line maps to +1 and the other maps to −1. Choose the inputs xn of the data set as random points (uniformly in X ), and evaluate the target function on each xn to get the corresponding output yn.

<b>5.</b> Take N = 100. Use Linear Regression to find g and evaluate Ein, the fraction of in-sample points which got classified incorrectly. Repeat the experiment 1000 times and take the average (keep the g’s as they will be used again in Problem 6). Which of the following values is closest to the average Ein? (Closest is the option that makes the expression |your answer −given option| closest to 0. Use this definition of closest here and throughout.)

<b>6.</b> Now, generate 1000 fresh points and use them to estimate the out-of-sample error Eout of g that you got in Problem 5 (number of misclassified out-of-sample points / total number of out-of-sample points). Again, run the experiment 1000 times and take the average. Which value is closest to the average Eout?</b>

<b>7.</b> Now, take N = 10. After finding the weights using Linear Regression, use them as a vector of initial weights for the Perceptron Learning Algorithm. Run PLA until it converges to a final vector of weights that completely separates all the in-sample points. Among the choices below, what is the closest value to the average number of iterations (over 1000 runs) that PLA takes to converge? (When implementing PLA, have the algorithm choose a point randomly from the set of misclassified points at each iteration)

In [6]:
class RandomLine:
    def __init__(self, pt1, pt2):
        self.pt1 = pt1
        self.pt2 = pt2
        self.slope = (pt1[1] - pt2[1]) / (pt1[0] - pt2[0])
        # y - y1 = m(x - x1) -> y = mx - mx1 + y1 
        self.intercept = self.slope * (-1 * pt1[0]) + pt1[1]
    
    def evaluate_pt(self, random_pt):
        # Evaluate if point is above or below line
        line_pt = random_pt[0]*self.slope + self.intercept
        if random_pt[1] > line_pt:
            return 1
        else:
            return -1

In [7]:
class PointsDF:
    def __init__(self, N, line):
        self.N = N
        self.line = line
        
        self.df = pd.DataFrame(np.random.uniform(-1,1,size=(self.N,2)))
        self.df['x_vector'] = list(zip(np.ones(self.df.shape[0]), self.df[0], self.df[1]))
        self.df['x_vector'] = self.df['x_vector'].apply(lambda x: np.asarray(x))
        self.df['y_initial'] = self.df['x_vector'].apply(lambda x: self.line.evaluate_pt(x[1:]))
           
    def linearReg(self):
        x = np.array(self.df['x_vector'].tolist())
        x_t = x.transpose()
        x_dagger = np.dot(np.linalg.inv(np.dot(x_t, x)), x_t)
        #x_dagger.shape
        y = np.array(self.df['y_initial'].tolist())
        weights = np.dot(x_dagger, y)
        return weights
    
    def evaluate(self, weights):
        self.df['y_predict'] = self.df['x_vector'].apply(lambda x: np.sign(np.dot(weights, x)))
        self.df['match'] = np.where(self.df.y_initial == self.df.y_predict, 1, 0)
        #match_pct = self.df['match'].sum() / self.df['match'].count()
        #return match_pct
    
    def PLA(self, weights):
        iters = 0
        self.evaluate(weights)
        matches = self.df['match'].sum()
        
        while matches != self.df.shape[0]: # While 1+ misclassified points
            iters = iters + 1

            # Use misclassified to update weights vector
            temp = self.df.query('match != 1').sample(1)
            weights = weights + temp['y_initial'].values[0]* temp['x_vector'].values[0]
            #print("Iter {} : {}".format(iters, weights))

            self.df['evaluate'] = self.df['x_vector'].apply(lambda x: np.sign(np.dot(weights, x)))
            self.df['match'] = np.where(self.df.y_initial == self.df.evaluate, 1, -1)
            matches = self.df['match'].sum()

        #print('Converged at Iteration {}'.format(iters))
        return iters, weights
        

In [8]:
OOS_N = 1000
results = []
OOS_results = []

for i in range(0,1000):
    #print("************ TRIAL {} ************".format(i))
    line = RandomLine(np.random.uniform(-1,1, 2), np.random.uniform(-1,1, 2))
    df = PointsDF(100, line)
    weights = df.linearReg()
    df.evaluate(weights)
    match_pct = df.df['match'].sum() / df.df['match'].count()
    results.append(match_pct)
    #print("In-sample Match Rate: {}".format(match_pct))
    
    # Out of Sample
    OOS = PointsDF(1000, line)
    OOS.evaluate(weights)
    OOS_match_pct = OOS.df['match'].sum() / OOS.df['match'].count()
    OOS_results.append(OOS_match_pct)
    #print("Out of Sample Match Rate: {}".format(OOS_match_pct), file=f)

In [9]:
print(1 - sum(results) / len(results))
print(1 - sum(OOS_results) / len(OOS_results))

0.03918999999999073
0.04748699999999961


<b> Answer (5) </b> <br></br>
<b>[c]</b> 0.01 <br></br>
<b> Answer (6) </b> <br></br>
<b>[c]</b> 0.01

In [10]:
## PLA initialized with weights from linear regression 
PLA_results = []
for i in range(0,1000):
    #print("************ TRIAL {} ************".format(i))
    line = RandomLine(np.random.uniform(-1,1, 2), np.random.uniform(-1,1, 2))
    test = PointsDF(10, line)
    weights = test.linearReg()
    test.evaluate(weights)
    iters, weights = test.PLA(weights)
    #print('Iterations: {}'.format(iters))
    PLA_results.append(iters)

In [11]:
sum(PLA_results) / len(PLA_results)

5.14

<b> Answer (7) </b> <br></br>
<b>[a]</b> 1

### Questions 8 - 10
<b>Nonlinear Transformation</b> <br></br>
In these problems, we again apply Linear Regression for classification. Consider the target function: <br></br>
f(x1, x2) = sign(x1^2 + x2^2 - 0.6)

Generate a training set of N = 1000 points on X = [−1, 1] × [−1, 1] with a uniform probability of picking each x ∈ X . Generate simulated noise by flipping the sign of the output in a randomly selected 10% subset of the generated training set.

<b>8.</b> Carry out Linear Regression without transformation, i.e., with feature vector: (1, x1, x2), to find the weight w. What is the closest value to the classification in-sample error Ein? (Run the experiment 1000 times and take the average Ein to reduce variation in your results.)

<b>9.</b> Now, transform the N = 1000 training data into the following nonlinear feature vector:
(1, x1, x2, x1x2, x1^2, x2^2). Find the vector ˜w that corresponds to the solution of Linear Regression. Which of the following hypotheses is closest to the one you find? Closest here means agrees the most with your hypothesis (has the highest probability of agreeing on a randomly selected point). Average over a few runs to make sure your answer is stable.

<b>10.</b> What is the closest value to the classification out-of-sample error Eout of your hypothesis from Problem 9? (Estimate it by generating a new set of 1000 points and adding noise, as before. Average over 1000 runs to reduce the variation in your results.)

In [12]:
class NoisyDF:
    def __init__(self, N):
        self.N = N
        self.line = line
        
        self.df = pd.DataFrame(np.random.uniform(-1,1,size=(self.N,2)))
        self.df['x_vector'] = list(zip(np.ones(self.df.shape[0]), self.df[0], self.df[1]))
        self.df['x_vector'] = self.df['x_vector'].apply(lambda x: np.asarray(x))
        
        self.df['y_initial'] = self.df['x_vector'].apply(lambda x: np.sign(pow(x[1],2) + pow(x[2], 2) - 0.6))
        self.df['rand'] = 1
        self.df['rand'] = self.df['rand'].apply(lambda x: x*np.random.rand())
        self.df['y_initial'] = np.where(self.df['rand'] > 0.1, self.df['y_initial'], self.df['y_initial']*-1)
           
    def linearReg(self, xVector_col):
        x = np.array(self.df[xVector_col].tolist())
        x_t = x.transpose()
        x_dagger = np.dot(np.linalg.inv(np.dot(x_t, x)), x_t)
        #x_dagger.shape
        y = np.array(self.df['y_initial'].tolist())
        weights = np.dot(x_dagger, y)
        return weights
    
    def evaluate(self, weights, x_vector_col):
        self.df['y_predict'] = self.df[x_vector_col].apply(lambda x: np.sign(np.dot(weights, x)))
        self.df['match'] = np.where(self.df.y_initial == self.df.y_predict, 1, 0)
        #match_pct = self.df['match'].sum() / self.df['match'].count()
        #return match_pct
        
    def x_transform(self):
        self.df['x_vector_t'] = self.df['x_vector'].apply(lambda x: np.array([1,x[1],x[2], x[1]*x[2], pow(x[1], 2), pow(x[2], 2)]))

In [13]:
noisy_results = []

for i in range(0,1000):
    df = NoisyDF(100)
    weights = df.linearReg('x_vector')
    df.evaluate(weights, 'x_vector')
    error = 1- (df.df['match'].sum() / df.df['match'].count())
    noisy_results.append(error)

In [14]:
error = 1 - (sum(noisy_results) / len(noisy_results))
print(error)

0.5583100000000002


<b> Answer (8) </b> <br></br>
<b>[d]</b> 0.5

In [15]:
df = NoisyDF(100)
df.x_transform()
weights_t = df.linearReg('x_vector_t')
df.evaluate(weights_t, 'x_vector_t')
print(weights_t)

[-0.95794898  0.29235185  0.01448495 -0.32265756  1.47330903  1.61461552]


In [16]:
## In Sample Error Rate for linear regression prediction
1 - (sum(df.df['match']) / len(df.df['match']))

0.09999999999999998

In [17]:
## Evaluate Hypothesis options
hypothesis = {'a': (np.array([-1, -0.05, 0.08, 0.13, 1.5, 1.5]), []),
'b' : (np.array([-1, -0.05, 0.08, 0.13, 1.5, 15]), []),
'c' : (np.array([-1, -0.05, 0.08, 0.13, 15, 1.5]), []),
'd' : (np.array([-1, -1.5, 0.08, 0.13, 0.05, 0.05]), []),
'e' : (np.array([-1, -0.05, 0.08, 1.5, 0.15, 0.15]), [])}

for i in range(0, 1000):
    #print("************ TRIAL {} ************".format(i))
    for h in hypothesis.keys():
        #print('HYPOTHESIS {}'.format(h))
        df.evaluate(hypothesis[h][0], 'x_vector_t')
        result = df.df['match'].sum() / df.df['match'].count()
        hypothesis[h][1].append(result)
        #print(result)

In [18]:
for i in hypothesis.keys():
    error = 1 - (sum(hypothesis[i][1]) / len(hypothesis[i][1]))
    print('{}: {}'.format(i, error))

a: 0.1299999999999959
b: 0.2999999999999936
c: 0.36000000000001076
d: 0.4199999999999998
e: 0.4599999999999934


<b> Answer (9) </b> <br></br>
<b>[a]</b> g(x1, x2) = sign(−1 − 0.05x1 + 0.08x2 + 0.13x1x2 + 1.5x1^2 + 1.5x2^2)

In [19]:
OOS_error = []
for i in range(0,1000):
    oos = NoisyDF(1000)
    oos.x_transform()
    oos.evaluate(hypothesis['a'][0], 'x_vector_t')
    error = 1 - (sum(oos.df['match']) / len(oos.df['match']))
    OOS_error.append(error)

In [20]:
sum(OOS_error) / len(OOS_error)

0.14303200000000016

<b> Answer (10) </b> <br></br>
<b>[b]</b> 0.1